# Spin pumping program
**`author`**: `Kuan-Chia Chiu`  
**`update`**: `2018-12-13`  
**`package installed`**: `pyvisa`, `serial`, `ipywidgets`

## Check available intruments
and only rum this cell once

In [3]:
import visa
import time
import serial
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
rm = visa.ResourceManager()
res = rm.list_resources()
vna_dict =[]
for x in range(0, len(res)):
    vna_dict.append(res[x])
    print("{}. ".format(x) + res[x] + ", \n")
print("\n")

measure_number = 1
graph_number = 1

0. ASRL4::INSTR, 

1. ASRL5::INSTR, 

2. ASRL6::INSTR, 

3. GPIB0::6::INSTR, 

4. GPIB0::16::INSTR, 





## GPIB ADDRESS

In [4]:
VNA_address = res[4]
KEPCO_address = res[3]
LakeShore_address = "COM6"
ser = serial.Serial(LakeShore_address, 57600,bytesize=7, parity=serial.PARITY_ODD, stopbits=1)
KEPCO = rm.open_resource(KEPCO_address)
vna = rm.open_resource(VNA_address)
print("Instrument Connected")

Instrument Connected


## Parameter Settings

In [27]:
#### Here start to give a current
sweep_current = 20
sweep_pts = 201
freq = 7 * 1000000000
IF_BAND = 10.000000
S_para = "S21"
delay = 1


#### Generate File name
import os
import datetime as date
year = date.datetime.now().year
day = date.datetime.now().day
month = date.datetime.now().month
data_Storage_PATH = 'C:\\Users\\KC\\Ipython notebook\\{}{}{}\\'.format(year, month, day)

if not os.path.exists(data_Storage_PATH):
    os.mkdir(data_Storage_PATH)

## DAQ

In [6]:
vna.timeout=10000
vna.write("SYST:PRESET")
vna.write("CALC:PAR:DEF 'My_{}', {}".format(S_para,S_para))
vna.write("DISPlay:WINDow1:STATe ON")
vna.write("DISPlay:WINDow1:TRACe1:FEED 'My_{}'".format(S_para))
vna.write("INITiate1:CONTinuous OFF;*OPC?")
opc = vna.read() # wait unitl complete

vna.write("SENSe1:SWEep:TRIGger:POINt OFF")
vna.write("SENSe1:SWEep:POINts {}".format(sweep_pts))
vna.write("SENSe1:SWEep:TIME {}".format(sweep_pts-1))
vna.write("SENSe1:BAND {}".format(IF_BAND))
vna.write("SENSe1:FREQuency:STARt {}".format(freq))
vna.write("SENSe1:FREQuency:STOP {}".format(freq))



KEPCO.timeout=10000
KEPCO.write("FUNC:MODE CURR")
KEPCO.write("volt 0.000;curr 0.000")
KEPCO.write("OUTPUT ON")
Field_data = []
Current_data = []
#### Start to run VNA and Power Supply
vna.write("INITiate1;")
for i in range(sweep_pts):
    KEPCO.write("volt  20.000;curr {:.4f}".format(i*sweep_current/(sweep_pts-1)))
    time.sleep(delay)
    Current = str(i*sweep_current/(sweep_pts-1))
    Current_data.append(Current)
    ser.write(b'RDGFIELD?\n')
    Field = ser.readline()
    Field_data.append(Field.decode('utf8'))
    time.sleep(0.1)

KEPCO.write("OUTPUT OFF")
KEPCO.write("volt 0.000;curr 0.000")
####

 
#Must select the measurement before we can read the data
vna.write("CALCulate1:PARameter:SELect 'My_S21'")
 
#Read the measurement data into the "result" string variable
vna.write("FORMat ASCII")
vna.write("CALCulate1:DATA? FDATA")
result = vna.read()
#print(result)


## Save Data to txt file

In [30]:
from pathlib import Path
measure_number = 1
data_str = "{}{}{}_{}_{}A_delay{}s_{:.0f}GHz_Meas_{:0>2d}".format(year,month,day,S_para,sweep_current,delay,freq/1000000000,measure_number)
my_file = Path(data_Storage_PATH+data_str+'.txt')
print(my_file)
while my_file.is_file():
    measure_number = measure_number + 1
    data_str = "{}{}{}_{}_{}A_delay{}s_{:.0f}GHz_Meas_{:0>2d}".format(year,month,day,S_para,sweep_current,delay,freq/1000000000,measure_number)
    my_file = Path(data_Storage_PATH+data_str+'.txt')
    
with open(data_str+'.txt', 'w') as f:
    f.write('Field\s(G)\t{}\s(G)\n'.format(S_para))
    for j in range(sweep_pts):
        f.write('{}\t{}\n'.format(Field_data[j][:-2],result.split(',')[j]))
f.close()

C:\Users\KC\Ipython notebook\20181213\20181213_S21_20A_delay1s_7GHz_Meas_01.txt
True
run
run


## Plotly Settings

In [10]:
import plotly as py
from plotly.graph_objs import Scatter, Layout
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go

##### change parameter #####
layout_title = '20181213_S21_test_our_CPW'
x_axis = 'Magnetic Field (G)'
y_axis = 'S21 (dB)'

############################

layout = go.Layout(
    margin = dict(l = 105, r = 50, b = 70, t = 50, pad = 4),
    title = layout_title,
    titlefont = dict(size=32),
    font = dict(size=16),
    xaxis=dict(
        title = x_axis,
        titlefont=dict(
            size=22,
        ),
        showgrid=True,
        zeroline=True,
        showline=True,
        mirror='ticks',
        gridcolor='#bdbdbd',
        gridwidth=2,
        zerolinecolor='#969696',
        zerolinewidth=4,
        linecolor='#636363',
        linewidth=6
    ),
    yaxis=dict(
        title = y_axis,
        titlefont=dict(
            size=22,
        ),
        showgrid=True,
        zeroline=True,
        showline=True,
        mirror='ticks',
        gridcolor='#bdbdbd',
        gridwidth=2,
        zerolinecolor='#969696',
        zerolinewidth=4,
        linecolor='#636363',
        linewidth=6
    )
)

In [11]:
trace = go.Scatter(x=Field_data, y=result.split(','), name='Original data')
iplot([trace], filename='test')


In [ ]:
import plotly.io as pio
graph_number = 1
graph_str = data_str + "Grap_{:0>2d}".format(graph_number)
my_file = Path(data_Storage_PATH + graph_str + '.pdf')
print(my_file)
while my_file.is_file():
    graph_number = graph_number + 1
    graph_str = data_str + "Grap_{:0>2d}".format(measure_number,graph_number)
    my_file = Path(data_Storage_PATH+graph_str + '.pdf')

fig = go.Figure(data=[trace], layout=layout)
pio.write_image(fig, graph_str +'.pdf')